In [1]:
# Install required packages
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import copy
import pandas as pd
from datetime import date

from scipy.optimize import minimize, check_grad, approx_fprime
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Import pyomo environment and setup gurobi solver
import pyomo.environ as pyo
from pyomo.opt import SolverStatus, TerminationCondition
import gurobipy 
solver = pyo.SolverFactory("gurobi_direct")

In [2]:
# Global variables
nr_items = 25
smple_sz = 1000
rsmpl_sz = 10
epsilon  = 10e-4
today = date.today()

# Implied variables
nr_trgts = nr_items + 1
nr_ftres = 2*nr_items
nr_ftres_intrcpt = nr_ftres + 1

In [3]:
# Set seeds
seed_lst = [*range(42, 42+rsmpl_sz)] 
np.random.seed(seed_lst[0])

In [4]:
# Source types: https://github.com/likr/kplib
# Type discription source: https://di.ku.dk/forskning/Publikationer/tekniske_rapporter/tekniske-rapporter-2003/03-08.pdf

ks_types = ['00Uncorrelated', '01WeaklyCorrelated', '02StronglyCorrelated', '03InverseStronglyCorrelated']
tp_insts = ['/s00'+str(i)+'.kp' for i in range(10)]

nr_typs  = len(ks_types)
nr_inst  = len(tp_insts)
f_list   = []

ins_b    = []
ins_v    = []
ins_w    = [] 

for ks_type in ks_types:
    for tp_inst in tp_insts:
        file_str = 'Instances/' + ks_type + tp_inst
        f = open(file_str, "r")
        f_list.append(f)
        i = 0
        lcl_v = []
        lcl_w = []
        for x in f:
            if i == 2:
                ins_b.append(int(x))
            if i > 3:
                x_split = x.split()
                lcl_v.append(int(x_split[0]))
                lcl_w.append(int(x_split[1]))
            i = i+1

        ins_v.append(lcl_v)
        ins_w.append(lcl_w)

In [5]:
print([x/ins_b[0] for x in ins_v[0]])
print([x/ins_b[0] for x in ins_w[0]])

[0.05717959128434159, 0.05129246176749222, 0.02848829340912167, 0.01752605223981594, 0.03464609554743538, 0.027405602923264313, 0.05305183380701042, 0.020571119231289754, 0.03227771010962241, 0.03951820273379348, 0.061510353227771014, 0.03417241845987278, 0.01908241981323589, 0.05115712545676005, 0.04188658817160644, 0.016984706996887266, 0.061578021383137095, 0.06651779672486129, 0.05487887400189471, 0.0611043442955745, 0.02104479631885235, 0.04939775341724185, 0.06083367167411016, 0.04628501827040195, 0.03200703748815807, 0.006834483691974556, 0.029435647584246855, 0.041345242928677764, 0.06184869400460143, 0.06543510623900392, 0.0323453782649885, 0.05860062254702937, 0.01766138855054811, 0.05454053322506428, 0.03714981729598051, 0.0010150223304912708, 0.048721071863580996, 0.026999593991067803, 0.0558262281770199, 0.04526999593991068, 0.00013533631073216944, 0.03342806875084585, 0.05873595885776154, 0.01651102990932467, 0.02205981864934362, 0.05893896332385979, 0.012992285830288266,

We consider the continuous knapsack problem, i.e.,
$$
\max c^{\top}z\\
s.t. Az\leq b\\
z\geq 0
$$
Here, $z$ is the decision vector with dimension $n$, nr of items. $c$ is the vector containing all the values of the items, $A$ is the vector containing the weight of the items, and wlog we set $b=1$.
To create explanations $c, A$ are used as features here. Hence there are $2n$ features.

In [6]:
# Continuous LP knapsack
def model_KS_cts(vals, output = 'goal', solver = solver):

    # Define a model
    model = pyo.ConcreteModel('Knapsack continuous model')

    # Declare decision variables
    model.x = pyo.Var(range(int(((len(vals)-1)/2))), domain=pyo.NonNegativeReals, bounds=(0, 1))

    # Declare objective
    model.objective = pyo.Objective(expr = sum(vals[i]*model.x[i] for i in range(int(((len(vals)-1)/2)))),
                                sense = pyo.maximize)

    # Declare constraints
    model.budget = pyo.Constraint(expr = sum(vals[i+int(((len(vals)-1)/2))]*model.x[i] for i in range(int(((len(vals)-1)/2)))) <= vals[-1])

    # Solve
    result = solver.solve(model)

    if output == 'goal':
        return model.objective()

    elif output == 'bounded' or output == 'feasibility':
        return result.solver.termination_condition != TerminationCondition.infeasibleOrUnbounded
    
    elif output == 'decision vector':
        solutions = []
        for i in range(int(((len(vals)-1)/2))):
            solutions.append(pyo.value(model.x[i]))
        return solutions
    
    elif output == 'all':
        solutions = []
        solutions.append(model.objective())
        for i in range(int(((len(vals)-1)/2))):
            solutions.append(pyo.value(model.x[i]))
        return solutions
    
    else:
        raise ValueError("Output not supported for model function")

In [9]:
ins_dict = {}
for i in range(nr_typs):
    ins_dict['Type '+str(i)] = {}
    for j in range(nr_inst):
        indx = int(10*i + j)
        ins_lcl_v = [x/ins_b[indx]*2 for x in ins_v[indx][:nr_items]]
        ins_lcl_w = [x/ins_b[indx]*2 for x in ins_w[indx][:nr_items]]
        print(sum(ins_lcl_w))
        ins_lcl_b = [1.0]
        ins_dict['Type '+str(i)]['Instance '+str(j)] = {}
        ins_dict['Type '+str(i)]['Instance '+str(j)]['c,A,b'] = np.concatenate((ins_lcl_v, ins_lcl_w, ins_lcl_b))

features = [*range(nr_ftres)] 

1.751522533495737
1.8502720988380643
2.201448225923244
2.281406295969403
1.7669134102255122
2.05015936923335
1.858186648953836
2.281883380106018
2.1108784706417842
2.030991555671629
2.159561766978018
2.0234037140676673
1.847310243183493
1.896170728015741
2.0128842860637692
2.1408915795266923
2.015457342237563
1.865024818633066
1.754863813229572
2.094499815430048
2.159561766978018
2.0234037140676673
1.847310243183493
1.896170728015741
2.0128842860637692
2.1408915795266923
2.015457342237563
1.865024818633066
1.754863813229572
2.094499815430048
2.1389254517171232
2.0228483319315953
1.8738626448959241
1.915615041427661
2.0141131768218212
2.12123206528893
2.0162004809517784
1.894679947494402
1.8007973700776387
2.080685147622267


We sample $m=1000$ instances of the knapsack problem. Let $\bold{X}\in\mathbb{R}^{m\times 2n+1}$ denote the full matrix of samples, then $X_{i}=(X_{i1},..., X_{i(2n)}, 1)$ is the $i$-th sample consisting of samples of all $2n$ features and 1 to ensure a value to multiply the intercept with.

Our output vecotr is denoted by $\bold{Y}\in\mathbb{R}^{m\times n+1}$ We want to explain multiple things $y_i=(c_i^{\top}z_i^*, z_i^*)$ is the $n+1$ dimensional output vector with $z_i^*$ the optimal decision vector for sample $i$. We set $y_{ij}=c_i^{\top}z_i^*$ for $j =0$ and $z_{ij}^*$ for $1\leq j\leq n$.

Moreover, we calculate the weight of eacht istance using the rbf-kernel. Set $W\in\mathbb{R}^{m\times m}$ and let $W_{ii}=W_i$ denote weight of $i$-th sample and set $W_{ij}=0$ when $i\neq j$.

In [10]:
# Help functions

# Create samples
def sample_perturbations_normal(orig, ftr_index_list, model_lcl, hyperprm = {}, mean = 0, var = 0.2, size = 1000, feasibility_check = True, bounded_check = True):
    
    org_plus_prtb = [orig]
    cntr = 1
    incr = 1

    while cntr < size:
        orig_with_noise = copy.deepcopy(orig)
        good_sample = True
        
        for j in range(len(orig)):
            if j in ftr_index_list:
                lcl_var = orig_with_noise[j] * var
                orig_with_noise[j] = orig_with_noise[j] + np.random.normal(mean, lcl_var)

        if feasibility_check:
            good_sample = good_sample * model_lcl(orig_with_noise, output = 'feasibility', **hyperprm)
        if bounded_check:
            good_sample = good_sample * model_lcl(orig_with_noise, output = 'bounded', **hyperprm)

        if good_sample == True:
            org_plus_prtb.append(np.asarray(orig_with_noise))
            cntr  = cntr + 1
        
        incr = incr + 1
        
    org_plus_prtb = np.asarray(org_plus_prtb)

    return org_plus_prtb

# Get values from samples
def get_values_from_samples(smpls, model_lcl, hyperprm = {}):
    values = []
    
    for smpl in smpls:
        values.append(model_lcl(smpl, **hyperprm))
    
    return values

# Determine weight of samples
def std_weight_function(a, b, ftr_index_list, kernel_width = None):
    d = np.linalg.norm(a - b)
    if kernel_width is None:
        krnl_wdth = 0.75 * len(ftr_index_list)
    else:
        krnl_wdth = kernel_width
    return np.exp(-(d ** 2) / (2* krnl_wdth ** 2))
    
def get_weights_from_samples(smpls, ftr_index_list, function = None, width = None):
    
    org = smpls[0]
    weights = []

    for smpl in smpls:
        if function is not None:
            weights.append(function(org, smpl))
        else:
            weights.append(std_weight_function(org, smpl, ftr_index_list, width))

    return weights

def get_knn(weights, k):
    return sorted(range(len(weights)), key=lambda i: weights[i])[-k:]

In [11]:
ins_dict['Type 1'].keys()

dict_keys(['Instance 0', 'Instance 1', 'Instance 2', 'Instance 3', 'Instance 4', 'Instance 5', 'Instance 6', 'Instance 7', 'Instance 8', 'Instance 9'])

In [12]:
for ky1 in ins_dict.keys():
    for ky2 in ins_dict[ky1].keys():
        print(ky1,ky2)
        for i in range(rsmpl_sz):
            np.random.seed(seed_lst[i])
            b = 'Batch '+ str(i)
            ins_dict[ky1][ky2][b] = {}
            # Sample new instances
            samples = sample_perturbations_normal(ins_dict[ky1][ky2]['c,A,b'], features, model_lcl = model_KS_cts, size=smple_sz)
            ins_dict[ky1][ky2][b]['smpl_vls'] = samples[:,features]
            ins_dict[ky1][ky2][b]['Samples']  = np.concatenate((ins_dict[ky1][ky2][b]['smpl_vls'],np.ones((smple_sz,1))), axis = 1) 

            # Find output of samples
            ins_dict[ky1][ky2][b]['Actuals']  = get_values_from_samples(samples, model_KS_cts, hyperprm = {'output':'all'})

            # Find weights of samples
            d_list = []
            for x in samples:
                d_list.append(np.linalg.norm(samples[0] - x))

            ins_dict[ky1][ky2][b]['Weights'] = get_weights_from_samples(samples, features, width=np.mean(d_list))

Type 0 Instance 0
Type 0 Instance 1
Type 0 Instance 2
Type 0 Instance 3
Type 0 Instance 4
Type 0 Instance 5
Type 0 Instance 6
Type 0 Instance 7
Type 0 Instance 8
Type 0 Instance 9
Type 1 Instance 0
Type 1 Instance 1
Type 1 Instance 2
Type 1 Instance 3
Type 1 Instance 4
Type 1 Instance 5
Type 1 Instance 6
Type 1 Instance 7
Type 1 Instance 8
Type 1 Instance 9
Type 2 Instance 0
Type 2 Instance 1
Type 2 Instance 2
Type 2 Instance 3
Type 2 Instance 4
Type 2 Instance 5
Type 2 Instance 6
Type 2 Instance 7
Type 2 Instance 8
Type 2 Instance 9
Type 3 Instance 0
Type 3 Instance 1
Type 3 Instance 2
Type 3 Instance 3
Type 3 Instance 4
Type 3 Instance 5
Type 3 Instance 6
Type 3 Instance 7
Type 3 Instance 8
Type 3 Instance 9


Lastly we introduce a $\bm{\beta}\in\mathbb{R}^{n+1\times 2n+1}$ consisting of the model variables. Here $\beta_{jk}$ is the lin. regression variable corresponding to output $j$ and feature $k$. Note, that $\beta_{j(2n+1)}$ corresponds to the intercept variable of output $j$. This way $\bm{X\beta}\approx \bm{Y}$.

In [13]:
smpl_vls_intr = ins_dict['Type 0']['Instance 0']['Batch 1']['Samples']
y_vals = ins_dict['Type 0']['Instance 0']['Batch 1']['Actuals']
wght_vls = ins_dict['Type 0']['Instance 0']['Batch 1']['Weights']

We define the following loss function (lss_all):
\begin{align}
\mathcal{L}(\bm{\beta}) = &\sum_{i=1}^m W_i  \sum_{j=0}^n\left(Y_{ij}-\sum_{k=1}^{2n+1}X_{ik}\beta_{jk}\right)^2\\
&+\lambda_{obj}\sum_{i=1}^m W_i  \left(\sum_{k=1}^{2n+1}X_{ik}\beta_{0k}-\sum_{j=1}^n c_{ij} \sum_{k=1}^{2n+1}X_{ik}\beta_{jk}\right)^2\\
&+\lambda_{cns}\sum_{i=1}^m W_i  \max\left\{0,\sum_{j=1}^n A_{ij} \sum_{k=1}^{2n+1}X_{ik}\beta_{jk}-1\right\}
\end{align}
Note that in here for the knapsack problem, $c_{ij}=X_{ij}$ is the value of item $j$ in sample $i$, and $A_{ij}=X_{i(j+n)}$ is the weight of item $j$ in sample $i$.

We can provide the Jacobian of $\mathcal{L}$ as follows. (1) and (2) are differentiable. For (3), we can provide a subgradient yielding the following partial derivatives:
\begin{align*}
\frac{\partial\mathcal{L}}{\partial \beta_{0k}} = &\sum_{i=1}^m (-2X_{ik})W_i  \left(Y_{i0}-\sum_{k=1}^{2n+1}X_{ik}\beta_{0k}\right)\\
&+\lambda_{obj}\sum_{i=1}^m (2X_{ik})W_i  \left(\sum_{k=1}^{2n+1}X_{ik}\beta_{0k}-\sum_{j=1}^n c_{ij} \sum_{k=1}^{2n+1}X_{ik}\beta_{jk}\right)\\
\frac{\partial\mathcal{L}}{\partial \beta_{jk}} = &\sum_{i=1}^m (-2X_{ik})W_i  \left(Y_{ij}-\sum_{k=1}^{2n+1}X_{ik}\beta_{jk}\right)& j>0\\
&+\lambda_{obj}\sum_{i=1}^m (-2c_{ij}X_{ik})W_i  \left(\sum_{k=1}^{2n+1}X_{i0}\beta_{0k}-\sum_{j=1}^n c_{ij} \sum_{k=1}^{2n+1}X_{ik}\beta_{jk}\right)\\
&+\lambda_{cns}\sum_{i=1}^m (A_{ij}X_{ik})W_i  \cdot\left[\sum_{j=1}^n A_{ij} \sum_{k=1}^{2n+1}X_{ik}\beta_{jk}>1\right]_{boolean} 
\end{align*}


In [14]:
# Loss function is the sum of the standard prediction loss,
# the objective loss, and the contraint loss

def lss_all(beta, X = smpl_vls_intr, Y = y_vals, W = wght_vls, lgr_obj = 1, lgr_cns = 1, oneD=True, region='all'):
    if oneD == True:
        beta = beta.reshape(nr_ftres_intrcpt, nr_trgts)

    if region == 'all':
        indc = [*range(len(X))] 
    elif isinstance(region, float) or isinstance(region, int):
        indc = get_knn(weights=W, k =int(region *len(W) / 100))
    elif region == 'present problem':
        indc = [0]
        
    std_err = np.sum(np.matmul(np.diag(W), np.square(Y - np.matmul(X, beta))))
    obj_err = np.sum(np.matmul(np.diag(np.array(W)), np.square(np.matmul(X, beta)[:,0] - np.sum(np.multiply(np.matmul(X, beta)[:,1:], X[:,:nr_items]), axis=1)))[indc])
    cns_err = np.sum(np.matmul(np.diag(np.array(W)), np.maximum(np.zeros(len(W)),  np.sum(np.multiply(np.matmul(X, beta)[:,1:], X[:,nr_items:-1]),axis=1)-np.ones(len(W))))[indc])

    tot_err = std_err + lgr_obj * obj_err + lgr_cns * cns_err 
    return tot_err

# Functions for the three components of the loss function seperately
def lss_std(beta, X = smpl_vls_intr, Y = y_vals, W = wght_vls, oneD=True):
    if oneD == True:
        beta = beta.reshape(nr_ftres_intrcpt, nr_trgts)
    return np.sum(np.matmul(np.diag(W),np.square(Y - np.matmul(X, beta))))

def lss_obj(beta, X = smpl_vls_intr, W = wght_vls, oneD=True, Y = y_vals):
    if oneD == True:
        beta = beta.reshape(nr_ftres_intrcpt, nr_trgts)
    return np.sum(np.matmul(np.diag(W),np.square(np.matmul(X, beta)[:,0]-np.sum(np.multiply(np.matmul(X, beta)[:,1:],X[:,:nr_items]), axis=1))))

def lss_cns(beta, X = smpl_vls_intr, W = wght_vls, oneD=True, Y = y_vals):
    if oneD == True:
        beta = beta.reshape(nr_ftres_intrcpt, nr_trgts)
    return np.sum(np.matmul(np.diag(W),np.maximum(np.zeros(len(W)),  np.sum(np.multiply(np.matmul(X, beta)[:,1:],X[:,nr_items:-1]),axis=1)-np.ones(len(W)))))

# Jacobian using subgradient
def lss_all_jac_sg(beta, X = smpl_vls_intr, Y = y_vals, W = wght_vls, lgr_obj = 1, lgr_cns = 1, oneD=True, region='all'):
    grad = np.zeros((nr_ftres_intrcpt, nr_trgts))
    if oneD == True:
        beta = beta.reshape(nr_ftres_intrcpt, nr_trgts)

    if region == 'all':
        indc = [*range(len(X))] 
    elif isinstance(region, float) or isinstance(region, int):
        indc = get_knn(W, int(region *len(W) / 100))
    elif region == 'present problem':
        indc = [0]

    tmp_std = (Y - np.matmul(X, beta))
    tmp_obj = (np.matmul(X, beta)[:,0]-np.sum(np.multiply(np.matmul(X, beta)[:,1:],X[:,:nr_items]), axis=1))
    tmp_cns = ((np.sum(np.multiply(np.matmul(X, beta)[:,1:],X[:,nr_items:-1]),axis=1)>np.ones(len(W))).astype(int))  


    for k in range(len(grad)):
        for j in range(len(grad[0])):
            grad[k][j] = grad[k][j] - 2 * (sum(W[i] * X[i][k] * tmp_std[i][j] for i in range(len(X))))
            if j == 0:
                grad[k][j] = grad[k][j] + 2 * lgr_obj * (sum(W[i] * X[i][k] * tmp_obj[i] for i in indc))
                pass
            else:
                grad[k][j] = grad[k][j] - 2 * lgr_obj * (sum(W[i] * X[i][k] * X[i][j-1] * tmp_obj[i] for i in indc))
                grad[k][j] = grad[k][j] + lgr_cns * (sum(W[i] * tmp_cns[i] * X[i][k] * X[i][j+nr_items-1] for i in indc))
                pass
    grad_flat = grad.flatten()
    return grad_flat

In [15]:
# Functions for the three components of the loss function seperately with predictions
def lss_std_pred(X = smpl_vls_intr, Y = y_vals, W = wght_vls, P=np.zeros(np.shape(y_vals))):
    return np.sum(np.matmul(np.diag(W),np.square(Y - P)))

def lss_obj_pred(X = smpl_vls_intr, Y = y_vals, W = wght_vls, P=np.zeros(np.shape(y_vals))):
    return np.sum(np.matmul(np.diag(W),np.square(P[:,0]-np.sum(np.multiply(P[:,1:],X[:,:nr_items]), axis=1))))

def lss_cns_pred(X = smpl_vls_intr, Y = y_vals, W = wght_vls, P=np.zeros(np.shape(y_vals))):
    return np.sum(np.matmul(np.diag(W),np.maximum(np.zeros(len(W)),  np.sum(np.multiply(P[:,1:],X[:,nr_items:-1]),axis=1)-np.ones(len(W)))))

In [16]:
# For each output j we fit a seperate linear regression model. 
# The variables of these models function as an initial beta, 
# a warm start to minimize the loss function later on.
def get_warm_start_linreg(x, y, w):
    models = []
    for i in range(len(y[0])):
        clf = LinearRegression()
        y_loc = [x[i] for x in y] 
        clf.fit(x, y_loc, sample_weight = w)
        models.append(clf)

    wrm_strt_t = np.ones((len(y[0]), len(features)+1))
    for i in range(len(models)):
        for j in range(len(features)):
            wrm_strt_t[i][j] = models[i].coef_[j]
        wrm_strt_t[i][j+1] = models[i].intercept_
    wrm_strt = np.transpose(wrm_strt_t)
    return wrm_strt

for ky1 in ins_dict.keys():
    for ky2 in ins_dict[ky1].keys():
        b = 'Batch 0'
        wrm_strt_lcl  = get_warm_start_linreg(x = ins_dict[ky1][ky2][b]['smpl_vls'], y = ins_dict[ky1][ky2][b]['Actuals'], w = ins_dict[ky1][ky2][b]['Weights'])
        print(ky1,ky2)
        # print(ins_dict[key]['wrm_strt '+str(1)])
        print(lss_std(wrm_strt_lcl, X = ins_dict[ky1][ky2][b]['Samples'], Y = ins_dict[ky1][ky2][b]['Actuals'], W =ins_dict[ky1][ky2][b]['Weights']))
        print(lss_obj(wrm_strt_lcl, X = ins_dict[ky1][ky2][b]['Samples'], Y = ins_dict[ky1][ky2][b]['Actuals'], W =ins_dict[ky1][ky2][b]['Weights']))
        print(lss_cns(wrm_strt_lcl, X = ins_dict[ky1][ky2][b]['Samples'], Y = ins_dict[ky1][ky2][b]['Actuals'], W =ins_dict[ky1][ky2][b]['Weights']), '\n')


Type 0 Instance 0
641.4265576153322
0.32415945478072994
4.95485576001823 

Type 0 Instance 1
254.12046332898916
0.045708033187807556
3.0862424776525255 

Type 0 Instance 2
634.7044296081741
0.3169982632199272
6.533657840533486 

Type 0 Instance 3
541.3744462380722
0.38181593002767356
7.427280649266891 

Type 0 Instance 4
210.962214548214
0.01658293600957903
3.0701256115794573 

Type 0 Instance 5
360.732229064059
0.5091397087499276
6.72327705216243 

Type 0 Instance 6
390.43572510087154
0.3071660534581578
6.112572265161637 

Type 0 Instance 7
344.6759667684256
0.10570406108839728
4.441308871210364 

Type 0 Instance 8
520.827241892191
0.15158445081560576
5.264841334386356 

Type 0 Instance 9
461.67552528604364
0.2411428036952301
7.122244383725182 

Type 1 Instance 0
1255.504909760713
1.0461268421250596
10.120479292782424 

Type 1 Instance 1
1012.3040069767322
1.0932306059770471
10.317964277001586 

Type 1 Instance 2
1109.61297937817
0.8846118334584769
8.790927197237451 

Type 1 Instance 

In [17]:
def lss_all_lmb(input, lmb_1, lmb_2):
    return lss_all(beta = input, lgr_obj=lmb_1, lgr_cns=lmb_2)

def lss_all_jac_lmb(input, lmb_1, lmb_2):
    return lss_all_jac_sg(beta = input, lgr_obj=lmb_1, lgr_cns=lmb_2)

In [18]:
def lss_all_input(input, X, Y, W, lmb_1, lmb_2, region):
    return lss_all(beta = input, X=X, Y=Y, W=W, lgr_obj=lmb_1, lgr_cns=lmb_2, region=region)

def lss_all_jac_input(input, X, Y, W, lmb_1, lmb_2, region):
    return lss_all_jac_sg(beta = input, X=X, Y=Y, W=W, lgr_obj=lmb_1, lgr_cns=lmb_2, region=region)

In [19]:
b = 'Batch '+ str(0)
X_lcl = ins_dict['Type 0']['Instance 0'][b]['Samples']
Y_lcl = ins_dict['Type 0']['Instance 0'][b]['Actuals']
W_lcl = ins_dict['Type 0']['Instance 0'][b]['Weights']
wrm_start_lcl = get_warm_start_linreg(x = X_lcl[:,features], y = Y_lcl, w = W_lcl).flatten()
args_RLR = (X_lcl, Y_lcl, W_lcl, 100, 10, 'all')
#Check grad provided
sol_RLR_lcl = minimize(lss_all_input, wrm_start_lcl, args = args_RLR, method='SLSQP', jac=lss_all_jac_input, options = {'maxiter': 1000})
print(sol_RLR_lcl)

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 671.4011729934911
       x: [ 7.474e-01  9.230e+00 ...  1.882e+00  1.662e+00]
     nit: 260
     jac: [ 1.035e-03  6.011e-01 ...  4.113e+00  2.911e+00]
    nfev: 730
    njev: 260


In [21]:
# sol_RLR_lcl_wo = minimize(lss_all_input, wrm_start_lcl, args = args_RLR, method='SLSQP', options = {'maxiter': 1000})
# print(sol_RLR_lcl_wo)

In [22]:
dict_methods_standard = {'Standard Linear Regression': {}, 
                        'Decision Tree Regression': {'max_depth': 5, 'min_samples_leaf': max(int(0.05*smple_sz), 50)}, 
                        'Regularized Linear Regression': {'Region': ['all']}}
dict_methods = {}
b = 'Batch 0'
for ky1 in ins_dict.keys():
    dict_methods[ky1] = {}
    for ky2 in ins_dict[ky1].keys():
        wrm_strt_lcl  = get_warm_start_linreg(x = ins_dict[ky1][ky2][b]['smpl_vls'], y = ins_dict[ky1][ky2][b]['Actuals'], w = ins_dict[ky1][ky2][b]['Weights'])
        lcl_lss_std = lss_std(wrm_strt_lcl, X = ins_dict[ky1][ky2][b]['Samples'], Y = ins_dict[ky1][ky2][b]['Actuals'], W =ins_dict[ky1][ky2][b]['Weights'])
        lcl_lss_obj = max(lss_obj(wrm_strt_lcl, X = ins_dict[ky1][ky2][b]['Samples'], Y = ins_dict[ky1][ky2][b]['Actuals'], W =ins_dict[ky1][ky2][b]['Weights']),10e-4)
        lcl_lss_cns = max(lss_cns(wrm_strt_lcl, X = ins_dict[ky1][ky2][b]['Samples'], Y = ins_dict[ky1][ky2][b]['Actuals'], W =ins_dict[ky1][ky2][b]['Weights']),10e-4)

        dict_methods[ky1][ky2] = copy.deepcopy(dict_methods_standard)
        # print(lcl_lss_std)
        # print(lcl_lss_obj)
        # print(lcl_lss_cns)
        # print((np.round(0.5*lcl_lss_std/lcl_lss_obj,decimals=1), 10), (np.round(0.5*lcl_lss_std/lcl_lss_obj, decimals=1),1))
        dict_methods[ky1][ky2]['Regularized Linear Regression']['Lambdas'] = [(np.round(0.5*lcl_lss_std/lcl_lss_obj,decimals=1), np.round(0.5*lcl_lss_std/lcl_lss_cns,decimals=1))]
                                                                            #   , (np.round(0.5*lcl_lss_std/lcl_lss_obj,decimals=1), 10), (np.round(0.5*lcl_lss_std/lcl_lss_obj, decimals=1),1)]

In [23]:
print(dict_methods)

{'Type 0': {'Instance 0': {'Standard Linear Regression': {}, 'Decision Tree Regression': {'max_depth': 5, 'min_samples_leaf': 50}, 'Regularized Linear Regression': {'Region': ['all'], 'Lambdas': [(np.float64(989.4), np.float64(64.7))]}}, 'Instance 1': {'Standard Linear Regression': {}, 'Decision Tree Regression': {'max_depth': 5, 'min_samples_leaf': 50}, 'Regularized Linear Regression': {'Region': ['all'], 'Lambdas': [(np.float64(2779.8), np.float64(41.2))]}}, 'Instance 2': {'Standard Linear Regression': {}, 'Decision Tree Regression': {'max_depth': 5, 'min_samples_leaf': 50}, 'Regularized Linear Regression': {'Region': ['all'], 'Lambdas': [(np.float64(1001.1), np.float64(48.6))]}}, 'Instance 3': {'Standard Linear Regression': {}, 'Decision Tree Regression': {'max_depth': 5, 'min_samples_leaf': 50}, 'Regularized Linear Regression': {'Region': ['all'], 'Lambdas': [(np.float64(708.9), np.float64(36.4))]}}, 'Instance 4': {'Standard Linear Regression': {}, 'Decision Tree Regression': {'max

In [24]:
for ky1 in ins_dict.keys():
    for ky2 in ins_dict[ky1].keys():
        print(ky1,ky2)
        for i in range(rsmpl_sz):
            b = 'Batch '+ str(i)
            print(b)
            X_lcl = np.array(ins_dict[ky1][ky2][b]['Samples'])
            Y_lcl = np.array(ins_dict[ky1][ky2][b]['Actuals'])
            W_lcl = np.array(ins_dict[ky1][ky2][b]['Weights'])
            wrm_start_lcl = get_warm_start_linreg(x = X_lcl[:,features], y = Y_lcl, w = W_lcl).flatten()

            for method in dict_methods[ky1][ky2].keys():
                ins_dict[ky1][ky2][b][method] = {}
                if method == 'Standard Linear Regression':
                    print('Standard Linear Regression')
                    ins_dict[ky1][ky2][b][method]['Beta'] = np.transpose(wrm_start_lcl.reshape(nr_ftres_intrcpt, nr_trgts))

                    for j in range(nr_trgts):
                        ins_dict[ky1][ky2][b][method]['Contribution Target ' + str(j)] = np.multiply(ins_dict[ky1][ky2][b][method]['Beta'][j,:-1], ins_dict[ky1][ky2]['c,A,b'][features])

                    ins_dict[ky1][ky2][b][method]['Error Prediction'] = lss_std(wrm_start_lcl, X = X_lcl, Y = Y_lcl, W = W_lcl)
                    ins_dict[ky1][ky2][b][method]['Error Objective']  = lss_obj(wrm_start_lcl, X = X_lcl, Y = Y_lcl, W = W_lcl)
                    ins_dict[ky1][ky2][b][method]['Error Constraint'] = lss_cns(wrm_start_lcl, X = X_lcl, Y = Y_lcl, W = W_lcl)

                elif method == 'Regularized Linear Regression':
                    print('Regularized Linear Regression')
                    for rgn in dict_methods[ky1][ky2][method]['Region']:
                        for lmbds in dict_methods[ky1][ky2][method]['Lambdas']:
                            ins_dict[ky1][ky2][b][method] = {}

                            args_RLR = (X_lcl, Y_lcl, W_lcl, lmbds[0], lmbds[1], rgn)
                            sol_RLR_lcl = minimize(lss_all_input, wrm_start_lcl, args = args_RLR, method='SLSQP', jac = lss_all_jac_input, options = {'maxiter': 1000}).x
                            ins_dict[ky1][ky2][b][method]['Beta'] = np.transpose(sol_RLR_lcl.reshape(nr_ftres_intrcpt, nr_trgts))
                            for j in range(nr_trgts):
                                ins_dict[ky1][ky2][b][method]['Contribution Target ' + str(j)] = np.multiply(ins_dict[ky1][ky2][b][method]['Beta'][j,:-1], ins_dict[ky1][ky2]['c,A,b'][features])  
                            
                            ins_dict[ky1][ky2][b][method]['Error Prediction'] = lss_std(sol_RLR_lcl, X = X_lcl, Y = Y_lcl, W = W_lcl)
                            ins_dict[ky1][ky2][b][method]['Error Objective']  = lss_obj(sol_RLR_lcl, X = X_lcl, Y = Y_lcl, W = W_lcl)
                            ins_dict[ky1][ky2][b][method]['Error Constraint'] = lss_cns(sol_RLR_lcl, X = X_lcl, Y = Y_lcl, W = W_lcl)

                elif method == 'Decision Tree Regression':
                    print('Decision Tree Regression')
                    hpr_prm = dict_methods[ky1][ky2][method]
                    ins_dict[ky1][ky2][b][method]['Beta'] = np.transpose(wrm_start_lcl.reshape(nr_ftres_intrcpt, nr_trgts))
                    Y_pred = []
                    for j in range(nr_trgts):
                        dtr = DecisionTreeRegressor(**hpr_prm)
                        dtr.fit(X_lcl[:,:-1], Y_lcl[:,j], sample_weight = W_lcl)
                        Y_pred.append(dtr.predict(X_lcl[:,:-1]))
                        ins_dict[ky1][ky2][b][method]['Contribution Target ' + str(j)] = dtr.feature_importances_
                    Y_pred = np.transpose(np.array(Y_pred))
                    ins_dict[ky1][ky2][b][method]['Error Prediction'] = lss_std_pred(X = X_lcl, Y = Y_lcl, W = W_lcl, P = Y_pred)
                    ins_dict[ky1][ky2][b][method]['Error Objective']  = lss_obj_pred(X = X_lcl, Y = Y_lcl, W = W_lcl, P = Y_pred)
                    ins_dict[ky1][ky2][b][method]['Error Constraint'] = lss_cns_pred(X = X_lcl, Y = Y_lcl, W = W_lcl, P = Y_pred)
                    
 


Type 0 Instance 0
Batch 0
Standard Linear Regression
Decision Tree Regression
Regularized Linear Regression
Batch 1
Standard Linear Regression
Decision Tree Regression
Regularized Linear Regression
Batch 2
Standard Linear Regression
Decision Tree Regression
Regularized Linear Regression
Batch 3
Standard Linear Regression
Decision Tree Regression
Regularized Linear Regression
Batch 4
Standard Linear Regression
Decision Tree Regression
Regularized Linear Regression
Batch 5
Standard Linear Regression
Decision Tree Regression
Regularized Linear Regression
Batch 6
Standard Linear Regression
Decision Tree Regression
Regularized Linear Regression
Batch 7
Standard Linear Regression
Decision Tree Regression
Regularized Linear Regression
Batch 8
Standard Linear Regression
Decision Tree Regression
Regularized Linear Regression
Batch 9
Standard Linear Regression
Decision Tree Regression
Regularized Linear Regression
Type 0 Instance 1
Batch 0
Standard Linear Regression
Decision Tree Regression
Regu

In [25]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [26]:
trgs = ['Contribution Target ' + str(i) for i in range(nr_trgts)]
ftrs = np.concatenate((['Val\n' + str(i+1) for i in range(nr_items)],['Wgt\n' + str(i+1) for i in range(nr_items)]))
print(trgs)

# Create dataframe to plot stability boxplots later on
df = pd.DataFrame.from_dict({(i,j,k,l): ins_dict[i][j][k][l]
                                for i in ins_dict.keys()
                                for j in ins_dict[i].keys()
                                for k in ['Batch '+ str(l) for l in range(rsmpl_sz)]
                                for l in dict_methods[i][j].keys()},
                            orient='index'
                            )
df.index.names = ['Type nr', 'Instance nr','Batch nr', 'Method']
df = df.reset_index()

df_lists = df

ftr_cols = []
for trg in trgs:
    trg_cols = np.concatenate(([trg + ' Val\n' + str(i+1) for i in range(nr_items)],[trg + ' Wgt\n' + str(i+1) for i in range(nr_items)]))
    ftr_cols = np.concatenate((ftr_cols,trg_cols))
    #split column of lists into new columns
    split = pd.DataFrame(df[trg].to_list(), columns = trg_cols)
    #join split columns back to original DataFrame
    df = pd.concat([df, split], axis=1) 
    #drop original column
    df = df.drop(trg, axis=1)
# explode_lst = list(trgs) + ['Features']
# df = df.explode(explode_lst, ignore_index=False)
df.to_excel("Overview_Original_df_25_items"+str(today)+".xlsx")  
display(df[df['Method']=='Standard Linear Regression'])

['Contribution Target 0', 'Contribution Target 1', 'Contribution Target 2', 'Contribution Target 3', 'Contribution Target 4', 'Contribution Target 5', 'Contribution Target 6', 'Contribution Target 7', 'Contribution Target 8', 'Contribution Target 9', 'Contribution Target 10', 'Contribution Target 11', 'Contribution Target 12', 'Contribution Target 13', 'Contribution Target 14', 'Contribution Target 15', 'Contribution Target 16', 'Contribution Target 17', 'Contribution Target 18', 'Contribution Target 19', 'Contribution Target 20', 'Contribution Target 21', 'Contribution Target 22', 'Contribution Target 23', 'Contribution Target 24', 'Contribution Target 25']


,Type nr,Instance nr,Batch nr,Method,Beta,Error Prediction,Error Objective,Error Constraint,Contribution Target 0 Val\n1,Contribution Target 0 Val\n2,...,Contribution Target 25 Wgt\n16,Contribution Target 25 Wgt\n17,Contribution Target 25 Wgt\n18,Contribution Target 25 Wgt\n19,Contribution Target 25 Wgt\n20,Contribution Target 25 Wgt\n21,Contribution Target 25 Wgt\n22,Contribution Target 25 Wgt\n23,Contribution Target 25 Wgt\n24,Contribution Target 25 Wgt\n25
0,Type 0,Instance 0,Batch 0,Standard Linear Regression,"[[0.7277117562999531, 0.916822078309722, 0.930...",641.426558,0.324159,4.954856,0.083221,0.094052,...,-0.039456,0.047251,-0.107175,-0.032991,-0.058021,-0.042201,0.002343,-0.022526,-0.021313,-1.221870
3,Type 0,Instance 0,Batch 1,Standard Linear Regression,"[[0.7632954709436864, 1.026018785736818, 0.992...",645.520956,0.337653,5.051188,0.087290,0.105254,...,0.008684,-0.007687,-0.057737,-0.008522,0.030135,-0.114816,0.008336,-0.033735,0.040701,-1.151329
6,Type 0,Instance 0,Batch 2,Standard Linear Regression,"[[0.6994250167326957, 1.0291789906086368, 1.00...",645.265557,0.303135,5.219517,0.079986,0.105578,...,-0.046989,-0.050062,-0.058246,-0.004238,0.008884,-0.020454,-0.039416,-0.067475,0.055180,-1.142082
9,Type 0,Instance 0,Batch 3,Standard Linear Regression,"[[0.7197691024953152, 0.9805081083806637, 1.08...",638.926732,0.326678,4.897391,0.082312,0.100585,...,-0.069115,-0.084675,-0.044433,0.018582,-0.079871,-0.005295,0.013701,0.000329,0.093729,-1.195320
12,Type 0,Instance 0,Batch 4,Standard Linear Regression,"[[0.7628378021730651, 1.0083804119903377, 0.93...",647.660678,0.329613,5.133511,0.087238,0.103445,...,0.097047,-0.209941,-0.042993,-0.026468,0.000608,0.036349,-0.046130,-0.084086,0.087641,-1.254962
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1185,Type 3,Instance 9,Batch 5,Standard Linear Regression,"[[0.49659716029557777, 0.4425064350133267, 0.3...",1062.876245,0.789718,9.427059,0.034621,0.024866,...,0.023242,-0.048245,-0.080472,0.002785,0.002518,-0.017658,0.015944,0.022788,-0.065025,-0.506730
1188,Type 3,Instance 9,Batch 6,Standard Linear Regression,"[[0.5043007155127691, 0.39774674648989383, 0.3...",1064.303519,0.733586,9.880854,0.035158,0.022351,...,0.000924,0.045567,-0.026982,-0.016895,0.002265,-0.004626,0.077906,0.071395,-0.004335,-0.501006
1191,Type 3,Instance 9,Batch 7,Standard Linear Regression,"[[0.5821150939298224, 0.4964565617228369, 0.44...",1049.298608,0.811217,9.406992,0.040583,0.027898,...,0.049042,-0.035306,-0.001722,0.004710,0.004954,0.006627,0.006181,0.053872,-0.002025,-0.496578
1194,Type 3,Instance 9,Batch 8,Standard Linear Regression,"[[0.5669225844487371, 0.6100229439430204, 0.16...",1061.280688,0.716930,9.790247,0.039524,0.034280,...,-0.014869,-0.027454,0.017063,-0.001653,-0.027823,0.022326,0.035715,-0.004834,0.005354,-0.481380


In [27]:
# determine % agreement over the x instances in the order of the first-k relative feature importance of Linear Regression surrogate
def agreement(df_lcl, type, instance, method, target_index, total_batches = rsmpl_sz):
    overlap       = []
    srted_indices = []
    ctoff_indices = []

    for i in range(total_batches):
        df_tmp = df_lcl[(df_lcl['Type nr'] == type) & (df_lcl['Instance nr'] == instance) & (df_lcl['Method'] == method) & (df_lcl['Batch nr'] == 'Batch '+str(i))]
        srtd_ind_btch = np.argsort(np.abs(df_tmp['Contribution Target ' +str(target_index)].iloc[0]))[::-1]
        ctff_ind_btch = np.where(np.array(np.abs(df_tmp['Contribution Target ' +str(target_index)].iloc[0])) > epsilon)[0]
        srted_indices.append(srtd_ind_btch)
        ctoff_indices.append(ctff_ind_btch)

    for i in range(1, len(srted_indices[0]) + 1):
        ovrlp_cnt = 0
        max_ovrlp = 0
        for j1 in range(len(srted_indices)):
            st1 = srted_indices[j1][:i]
            co1 = ctoff_indices[j1]
            sc1 = list(set(st1) & set(co1))
            for j2 in range(j1+1, len(srted_indices)):
                st2 = srted_indices[j2][:i]
                co2 = ctoff_indices[j2]
                sc2 = list(set(st2) & set(co2))
                lcl_intrsctn  = list(set(sc1) & set(sc2))
                ovrlp_cnt = ovrlp_cnt + len(lcl_intrsctn)
                max_ovrlp = max_ovrlp + i
        if max_ovrlp == 0:
            overlap.append(1)
        else:
            overlap.append(ovrlp_cnt/max_ovrlp)
    
    return overlap

In [28]:
dict_methods['Type 0']['Instance 0'].keys()

dict_keys(['Standard Linear Regression', 'Decision Tree Regression', 'Regularized Linear Regression'])

In [29]:
f = 5

df['Agreement'] = np.nan
df['Agreement top '+str(f)] = np.nan
        
for i in ins_dict.keys():
    for j in ins_dict[i].keys():
        for k in dict_methods[i][j].keys():
            # print(i,j,k)
            agrm_mthd = 0
            agrm_mthd_f = 0
            for t in range(nr_trgts):
                lcl_agrm = agreement(df_lists, type=i, instance=j, method=k, target_index=t, total_batches= rsmpl_sz)
                agrm_mthd   = agrm_mthd   + sum(lcl_agrm)
                agrm_mthd_f = agrm_mthd_f + sum(lcl_agrm[:f])
            print(float(agrm_mthd/nr_trgts))

            df['Agreement'] = np.where((df['Type nr']==i) & (df['Instance nr']==j) & (df['Method']==k),float(agrm_mthd/nr_trgts), df['Agreement'])
            df['Agreement top '+str(f)] = np.where((df['Type nr']==i) & (df['Instance nr']==j) & (df['Method']==k),float(agrm_mthd_f/nr_trgts), df['Agreement top '+str(f)])

26.742602060288277
6.240664709792881
30.6016301878197
19.759575362592628
4.470045994015143
30.89313351527678
25.821759690251593
5.577092531882536
30.86682602778583
27.108044553620896
5.664564814145605
30.683578299837983
19.212677621429187
4.293612892674031
30.48914746866275
22.0335763771199
4.490257146347481
30.38524965928834
21.933887947621233
4.391497226779465
30.23865812921907
24.484807497875696
4.913849930667229
30.686271796294236
24.24825347401201
5.3928346569392165
31.16184825862525
20.547353808464525
4.5579372806383835
30.655095345329467
32.36784393782383
7.755825555890184
31.223848335451066
32.051986544238154
7.453676842947754
31.32349055372939
31.01132539774322
7.2656930248387575
31.188287226813788
31.364923055820313
7.0638577381935015
30.947861057511943
33.06687751333445
7.73881955363345
31.666802651869684
30.644055344732294
7.273256708068701
30.159570249275156
31.392757684632947
7.647718015553186
30.970155772438982
32.12841911978259
7.000668033852978
31.419173795796727
32.92

In [30]:
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

agg_dict = {'Error Prediction':'mean', 'Error Objective':'mean', 'Error Constraint':'mean', 'Agreement':'mean', 'Agreement top '+str(f):'mean'}
for ftr in ftr_cols:
    agg_dict[ftr] = ['mean', 'std']

df_group = df.groupby(['Type nr', 'Method']).agg(agg_dict)

top_cols_all = []
for trg in trgs:
    top_cols = [(trg, 'Top ' + str(i+1)) for i in range(f)]
    micolumns = pd.MultiIndex.from_tuples(top_cols)
    top_cols_all = np.concatenate((top_cols_all, micolumns))
    slc_cols = [(trg + ' ' + ftr, 'mean') for ftr in ftrs]
    tmp = df_group[slc_cols].apply(lambda x: np.absolute(x).nlargest(f).index.tolist(), axis=1).tolist()
    output = [[item[0] for item in lst] for lst in tmp]
    df_group = df_group.join(pd.DataFrame(output, 
                           columns=micolumns, index=df_group.index))

df_group[('Top '+str(f)+' contributors', '')] = df_group[top_cols_all].values.tolist()
df_group = df_group.drop(columns=top_cols_all)

ftr_sgn_lst = []
ftr_std_lst = []
ftr_nrm_std_lst = []
for ftr in ftr_cols:
    #Significant if mean contribution is larger than epsilon
    df_group[(ftr, 'sign.')] = np.where(np.absolute(df_group[(ftr,'mean')].astype(float)) <= epsilon, 0, 1)
    #In a top 5 contributor
    df_group[(ftr, 'top '+str(f))] = df_group[('Top '+str(f)+' contributors', '')].apply(lambda lst: 1 if ftr in lst else 0)
    #Index of dispersion
    df_group[(ftr, 'Normalized std')] = np.where(df_group[(ftr, 'sign.')]==0, 0, np.absolute(df_group[(ftr, 'std')]/df_group[(ftr, 'mean')]))
    
    ftr_sgn_lst.append((ftr, 'sign.'))
    ftr_std_lst.append((ftr, 'std'))
    ftr_nrm_std_lst.append((ftr, 'Normalized std'))


df_group['Avg. Normalized std'] = (sum(df_group[(i, 'sign.')]* df_group[(i, 'Normalized std')] for i in ftr_cols)/sum(df_group[(j, 'sign.')] for j in ftr_cols)).astype(float)
df_group['Std. Normalized std'] = (sum(df_group[(i, 'sign.')]*(df_group[(i, 'Normalized std')] - df_group['Avg. Normalized std'])**2  for i in ftr_cols)/sum(df_group[(j, 'sign.')] for j in ftr_cols)).astype(float)
df_group['Avg. std'] = (sum(df_group[(i, 'std')] for i in ftr_cols)/len(ftr_cols)).astype(float)
df_group['Std. std'] = (sum((df_group[(i, 'std')] - df_group['Avg. std'])**2  for i in ftr_cols)/len(ftr_cols)).astype(float)


df_group['Avg. Normalized std top '+str(f)] = (sum(df_group[(i, 'sign.')]*df_group[(i, 'top '+str(f))]*df_group[(i, 'Normalized std')] for i in ftr_cols)/sum(df_group[(j, 'sign.')]*df_group[(j, 'top '+str(f))] for j in ftr_cols)).astype(float)
df_group['Std. Normalized std top '+str(f)] = (sum(df_group[(i, 'sign.')]*df_group[(i, 'top '+str(f))]*(df_group[(i, 'Normalized std')] - df_group['Avg. Normalized std top '+str(f)])**2  for i in ftr_cols)/sum(df_group[(j, 'sign.')]*df_group[(j, 'top '+str(f))] for j in ftr_cols)).astype(float)
df_group['Avg. std top '+str(f)] = (sum(df_group[(i, 'sign.')]*df_group[(i, 'top '+str(f))]*df_group[(i, 'std')] for i in ftr_cols)/sum(df_group[(j, 'sign.')]*df_group[(j, 'top '+str(f))] for j in ftr_cols)).astype(float)
df_group['Std. std top '+str(f)] = (sum(df_group[(i, 'sign.')]*df_group[(i, 'top '+str(f))]*(df_group[(i, 'std')] - df_group['Avg. std'])**2  for i in ftr_cols)/sum(df_group[(j, 'sign.')]*df_group[(j, 'top '+str(f))] for j in ftr_cols)).astype(float)

In [34]:
col_dsply = [('Error Prediction','mean'),('Error Objective','mean'), ('Error Constraint','mean'), ('Avg. std',''), ('Std. std',''), ('Avg. Normalized std',''), ('Std. Normalized std',''), ('Avg. std top 5',''), ('Std. std top 5',''), ('Avg. Normalized std top 5',''), ('Std. Normalized std top 5',''), ('Agreement','mean'), ('Agreement top 5','mean')]
df_dsply = df_group[col_dsply]
# display(df_dsply.loc['Type 0'].style.background_gradient(axis=0))
display(df_dsply.loc['Type 3'])
# df_dsply.to_excel("Overview_test_25_items"+str(today)+".xlsx")  

,Error Prediction,Error Objective,Error Constraint,Avg. std,Std. std,Avg. Normalized std,Std. Normalized std,Avg. std top 5,Std. std top 5,Avg. Normalized std top 5,Std. Normalized std top 5,Agreement,Agreement top 5
,mean,mean,mean,,,,,,,,,mean,mean
Method,,,,,,,,,,,,,
Decision Tree Regression,893.373507,13.800736,37.736463,0.009389,0.001212,2.855253,10.630062,0.135511,0.019339,2.521614,14.455971,7.395157,2.994487
Regularized Linear Regression,1240.686027,0.267001,0.031802,0.064030,0.003110,2.877071,18.356301,0.157610,0.029367,0.768608,0.141918,31.131711,3.377201
Standard Linear Regression,1110.585571,0.700378,8.937026,0.062956,0.004720,1.701240,2.738628,0.176167,0.045077,0.663806,0.093595,31.384282,3.306058
